In [1]:
!pip install -Uq /kaggle/input/llm-whls/bitsandbytes-0.41.1-py3-none-any.whl
!pip install -Uq /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install -Uq /kaggle/input/library-off-for-llm/transformers-4.38.2-py3-none-any.whl

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch

from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
input_token_len = 1024
output_token_len = 100

In [4]:
test_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

In [5]:
base_model_name = "/kaggle/input/phi/transformers/2/1"
adapter_model_name = "/kaggle/input/phi2-public-data-sft-adapter/pytorch/public-data-sft/1/phi2_public_data_sft"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name,trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = AutoModelForCausalLM.from_pretrained(base_model_name,trust_remote_code=True)
model = PeftModel.from_pretrained(model, adapter_model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
model.to(device)
model.eval()
print('model loaded !!')

model loaded !!


In [10]:
def text_generate(ori_text, rew_text, model, tokenizer, input_max_len=512, output_len=20, device='cuda'):
    prompt = f"Instruct: Original Text:{ori_text}\nRewritten Text:{rew_text}\nWrite a prompt that was likely given to the LLM to rewrite original text to rewritten text.\nOutput:"
    inputs = tokenizer(prompt, max_length=input_max_len, truncation=True, return_tensors="pt", return_attention_mask=False)
    
    input_token_len = len(inputs.input_ids[0])
    inputs = {k:v.to(device) for k,v in inputs.items()}
    max_len = input_token_len + output_len
    outputs = model.generate(**inputs,
                         do_sample=False,
                         max_length=max_len,
                         pad_token_id=tokenizer.pad_token_id,
                         )
    text = tokenizer.batch_decode(outputs,skip_special_tokens=True,clean_up_tokenization_spaces=False)[0]
    start_index = text.find('Output:')
    generated_text = text[start_index+len('Output:'):].strip()
    return generated_text

In [11]:
mean_prompt = "'Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.'"

In [12]:
rewrite_prompts = []

In [13]:
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    prompt = mean_prompt
    try:
        prompt = text_generate(row['original_text'],
                               row['rewritten_text'],
                               model,
                               tokenizer,
                               input_token_len,
                               output_token_len,
                               device,
                              )
    except:
        pass
        
    rewrite_prompts.append(prompt)

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
test_df['rewrite_prompt'] = rewrite_prompts

In [15]:
sub_df = test_df[['id', 'rewrite_prompt']]
sub_df.to_csv('submission.csv', index=False)